In [ ]:

import visio as v

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import visio as v

def contarPunts(roi):
    dst = np.zeros((roi.shape[0],roi.shape[1],3),dtype=np.uint8)
    dst[:,:,0] = roi[:,:]
    dst[:,:,1] = roi[:,:]
    dst[:,:,2] = roi[:,:]
    dst = cv.GaussianBlur(dst,(5,5),0)
    _,threshold = cv.threshold(dst,127,255,cv.THRESH_BINARY)    
    contours, hierarchy = cv.findContours(threshold[:,:,0],cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    punts = 0
    
    dst = cv.drawContours(dst,contours,-1,(255,127,0),1)
    #plt.figure()
    #plt.imshow(dst,'gray')
    for i,c in enumerate(contours):
        pare = hierarchy[0][i,3]
        if pare != -1:
            punts+=1
    return punts

if __name__ == '__main__':
    j0 = cv.imread('src/jugada/00.jpg')
    j1 = cv.imread('src/jugada/01.jpg')
    j2 = cv.imread('src/jugada/02.jpg')
    j3 = cv.imread('src/jugada/03.jpg')
    vis = v.Visio()
    
    #it 1
    vis.updateFrame(j0)
    out = vis.getTableData()
    #plt.figure()
    #plt.imshow(out,'gray')
    #it 2
    vis.updateFrame(j1)
    out = vis.getTableData()
    #plt.figure()
    #plt.imshow(out,'gray')
    #it 3
    vis.updateFrame(j2)
    out = vis.getTableData()
    
    vis.updateFrame(j3)
    out = vis.getTableData()
    print("rotacio:{}".format(vis.rotacioDefecte))
    plt.figure()
    plt.imshow(vis.rotatedFrame,'gray')
    
    final = vis.rotatedFrame
    arrayPatrons = []
    arrayPatrons.append(vis.patroV)
    arrayPatrons.append(vis.patroH)
    
    zonaMatch = int(min(vis.patroV.shape))
    #arrayPatrons.append(np.flip(vis.patroV))
    #arrayPatrons.append(np.flip(vis.patroH))
    
    estatPartida={'maRobot':{},'maHuma':{},'taulell':{},'pou':{}}
    
    print('zonaMatch: {}'.format(zonaMatch))
    found=[]
    for patro in arrayPatrons:  
        
        w, h = patro.shape[::-1]
        res = cv.matchTemplate(final,patro,cv.TM_CCOEFF_NORMED) #TM_SQDIFF_NORMED
        thr = 0.8
        loc = np.where( res >= thr)
        same=False
        for pt in zip(*loc[::-1]):  
            if len(found)==0:
                same=False
            else:
                same=False
                for f in found:
                    if pt[0] in (np.arange(f[0]-zonaMatch,f[0]+zonaMatch)):
                        if pt[1] in (np.arange(f[1]-zonaMatch,f[1]+zonaMatch)):
                            same = True
                            break
            if not same:
                found.append(pt)  
                top_left = pt
                bottom_right = (pt[0] + w, pt[1] + h)
                
                top_right = (bottom_right[0],top_left[1])
                bottom_left = (top_left[0],bottom_right[1])
                
                center = (int(top_left[0]+(round((top_right[0]-top_left[0])/2))) ,int(top_left[1]+(round((bottom_left[1]-top_left[1])/2))))


                orientacio = 1 #h
                alcada = vis.midaFitxa[0]
                amplada = vis.midaFitxa[1]
                if top_right[0]-top_left[0] < bottom_left[1]-top_left[1]:
                    orientacio=0
                    alcada = vis.midaFitxa[1]
                    amplada = vis.midaFitxa[0]


                estatPartida['taulell'][len(estatPartida['taulell'])]=[(center[0],center[1],amplada,alcada),[0,0],orientacio]  
                final = cv.rectangle(final, pt, (pt[0] + w, pt[1] + h), (0,127,255), 2)
        
       
    plt.figure()
    plt.imshow(final,'gray')
    cv.imwrite('fina.jpg',final)
    
    
    #idFitxa : [ (x,y,amplada,alçada), [ puntsEsquerra/Dalt, puntsDreta/Baix], orientació]
    for dic in estatPartida['taulell']:
        #print(estatPartida['taulell'][dic])
        x,y,w,h = estatPartida['taulell'][dic][0]
        puntsA=0
        puntsB=0
        if estatPartida['taulell'][dic][2]: # Vertical
            # ROI
            roi = final[y-round(h/2) : y,x-round(w/2) : x+round(w/2)] # Superior
            puntsA=contarPunts(roi)
            roi = final[y : y+round(h/2),x-round(w/2) : x+round(w/2)] # Inferior
            puntsB=contarPunts(roi)

        else: # Horitzontal
            # ROI
            roi = final[y-round(h/2) : y+round(h/2),x-round(w/2) : x] # Esquerra
            puntsA=contarPunts(roi)
            
            roi = final[y-round(h/2) : y+round(h/2),x : x+round(w/2)] # Dreta
            puntsB=contarPunts(roi)
        
        estatPartida['taulell'][dic][1]=[puntsA,puntsB]
    
    for d in estatPartida['taulell']:
        print("{}: {}".format(d,estatPartida['taulell'][d]))
    # ProcessarFrame():
    # si hi ha orientacio rotarla
    # si no hi ha patroFitxa, buscar-lo
    # Si hi patroFitxa, patternmatching
        # Recompte de punts 
    


In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import visio as v

def contarPunts(roi):
    dst = np.zeros((roi.shape[0],roi.shape[1],3),dtype=np.uint8)
    dst[:,:,0] = roi[:,:]
    dst[:,:,1] = roi[:,:]
    dst[:,:,2] = roi[:,:]
    dst = cv.GaussianBlur(dst,(5,5),0)
    _,threshold = cv.threshold(dst,127,255,cv.THRESH_BINARY)    
    contours, hierarchy = cv.findContours(threshold[:,:,0],cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    punts = 0
    
    dst = cv.drawContours(dst,contours,-1,(255,127,0),1)
    #plt.figure()
    #plt.imshow(dst,'gray')
    for i,c in enumerate(contours):
        pare = hierarchy[0][i,3]
        if pare != -1:
            punts+=1
    return punts

if __name__ == '__main__':
    j0 = cv.imread('src/jugada/00.jpg')
    j1 = cv.imread('src/jugada/01.jpg')
    j2 = cv.imread('src/jugada/02.jpg')
    j3 = cv.imread('src/jugada/03.jpg')
    j4 = cv.imread('src/jugada/04.jpg')
    vis = v.Visio()
    
    #it 1
    vis.updateFrame(j0)
    estatPartida = vis.getTableData()
    vis.updateFrame(j1)
    estatPartida = vis.getTableData()
    vis.updateFrame(j2)
    estatPartida = vis.getTableData()
    vis.updateFrame(j3)
    estatPartida = vis.getTableData()


    plt.figure()
    plt.imshow(vis.rotatedFrame,'gray')
    out = vis.rotatedFrame
    for d in estatPartida['taulell']:
        print("{}: {}".format(d,estatPartida['taulell'][d]))
   


    # ProcessarFrame():
    # si hi ha orientacio rotarla
    # si no hi ha patroFitxa, buscar-lo
    # Si hi patroFitxa, patternmatching
        # Recompte de punts 
    
#x:-30 +30
#y:  0 +55

0: [(17.71875, 52.40625, 85, 45), [0, 0], 0]
1: [(1.6875, 38.59375, 85, 45), [4, 4], 0]
2: [(12.65625, 48.072916666666664, 45, 85), [0, 6], 1]
3: [(6.1875, 42.65625, 45, 85), [0, 4], 1]
4: [(-4.03125, 33.854166666666664, 45, 85), [4, 5], 1]


In [ ]:
estatPartida={
    'maRobot':{ 
            #idFitxa : [ (x,y,amplada,alçada), [ puntsEsquerra/Dalt, puntsDreta/Baix], orientació, extrem]
            0 :[(5,5,4,2),[1,1],0],
            1 :[(5,10,4, 2),[4,4],0],
            2 :[(5,15,4, 2),[4,2],0],
            3 :[(5,220,4, 2),[2,3],0]  
    },
    'maHuma':{ 
        0 :[(55,40,4,2),[1,2],0],
        1 :[(55,45,4, 2),[5,0],0] 
    },
    'taulell':{ 
        0 :[(25,23,4,2),[4,3],1],
        1 :[(23,25,4, 2),[6,3],0],
        2 :[(21,25,4, 2),[6,6],1]    
    },
    'pou':{
        
    },
    'extrems':{
        0 :[(25,23,4,2),[4,3],1],
        2 :[(21,25,4, 2),[6,6],1] 
    }
}
print(estatPartida)
print()
for m in estatPartida:
    print("{}:{}".format(m, estatPartida[m]))

In [ ]:

patroV = cv.cvtColor(cv.imread('patro.jpg'),cv.COLOR_BGR2GRAY)
patroH = patroV.transpose()

patroVI = np.flip(patroV)

patroHI = np.flip(patroH)


plt.figure()
plt.imshow(patroV,'gray')
plt.figure()
plt.imshow(patroH,'gray')
plt.figure()
plt.imshow(patroVI,'gray')
plt.figure()
plt.imshow(patroHI,'gray')